<img src="../img/Signet_FNW_1.svg" alt="OVGU_FNW_Logo" width="300" align="right">

# 1.1. Kinematics in one dimension
In this chapter, ...


## 1.1.1. TBA
... todo

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from ipywidgets import interact
import ipywidgets as widgets
from scipy.constants import pi, epsilon_0 


def coulomb(q1, q2, r):
    return (1 / (4 * pi * epsilon_0)) * q1 * q2 / r**2  # Coulomb's law


# define global variables
FORCE_LIST_Q = []
FORCE_LIST_R = []
Q1_LIST = []
Q2_LIST = []
R_LIST = []
Q_MIN = 0
Q_MAX = 1
R_MIN = 0.1
R_MAX = 1
FMAX = coulomb(Q_MAX, Q_MAX, R_MIN)


def plot_coulomb_q(q1, q2, elev, azim, n_trace, show_analytical):
    # compute the force
    force = coulomb(q1, q2, R_MIN)

    # append to global lists  
    if len(FORCE_LIST_Q) < n_trace:
        Q1_LIST.append(q1)
        Q2_LIST.append(q2)
        FORCE_LIST_Q.append(force)    
    else:
        if force != FORCE_LIST_Q[-1]: # hack for plotting / not losing scatters/ appending when rotating coordinate system
            Q1_LIST.pop(0)
            Q2_LIST.pop(0)
            FORCE_LIST_Q.pop(0)
            Q1_LIST.append(q1)
            Q2_LIST.append(q2)
            FORCE_LIST_Q.append(force)

    # initialize the plot
    fig = plt.figure(figsize=(6, 4))
    gs = gridspec.GridSpec(1, 1, width_ratios=[1], height_ratios=[1])
    ax_3d = fig.add_subplot(gs[0], projection='3d')
    # plot the analytical solution if requested
    if show_analytical:
        plane_q1, plane_q2 = np.meshgrid(np.linspace(Q_MIN, Q_MAX, 100), np.linspace(Q_MIN, Q_MAX, 100))
        plane_f = coulomb(plane_q1, plane_q2, R_MIN)
        ax_3d.plot_surface(plane_q1, plane_q2, plane_f, alpha=0.5, cmap=plt.cm.gray)
    # scatter plot the force
    ax_3d.scatter(Q1_LIST, Q2_LIST, FORCE_LIST_Q, s=5, alpha=1, c=range(len(FORCE_LIST_Q)), cmap=plt.cm.hsv) 
    
    # format the plot
    ax_3d.view_init(elev=elev, azim=azim, roll=0)
    ax_3d.set_xlim([Q_MIN, Q_MAX])
    ax_3d.set_ylim([Q_MIN, Q_MAX])
    ax_3d.set_zlim([0, FMAX])
    #ax_3d.set_aspect('equal')
    ax_3d.set_proj_type('ortho')
    ax_3d.set_xlabel('q1 [C]')
    ax_3d.set_ylabel('q2 [C]')
    ax_3d.set_zlabel('F [N]')

    plt.show()


# to transform the plot to a sqrt scale, we need to define a forward and inverse function
def forward(x):
    return x**(1/2)


def inverse(x):
    return x**(2)


def plot_coulomb_r(r, n_trace, show_analytical, transform_y_axis):
    # compute the force
    force = coulomb(Q_MAX, Q_MAX, r)

    # append to global lists  
    if len(FORCE_LIST_R) < n_trace:
        R_LIST.append(r)        
        FORCE_LIST_R.append(force)    
    else:
        if force != FORCE_LIST_R[-1]: # hack for plotting / not losing scatters/ appending when rotating coordinate system
            R_LIST.pop(0)
            FORCE_LIST_R.pop(0)
            R_LIST.append(r)
            FORCE_LIST_R.append(force)

    # initialize the plot
    fig = plt.figure(figsize=(3, 3))
    gs = gridspec.GridSpec(1, 1, width_ratios=[1], height_ratios=[1])
    ax_2d = fig.add_subplot(gs[0])
    # plot the analytical solution if requested
    if show_analytical:
        line_r = np.linspace(R_MIN, R_MAX, 100)
        line_f = coulomb(Q_MAX, Q_MAX, line_r)
        ax_2d.plot(line_r, line_f, alpha=0.5, c='gray')
    # scatter plot the force
    ax_2d.scatter(R_LIST, FORCE_LIST_R, s=15, alpha=1, c=range(len(FORCE_LIST_R)), cmap=plt.cm.hsv)     
    # format the plot
    if transform_y_axis:
        ax_2d.set_yscale('function', functions=(forward, inverse))
    else:
        ax_2d.set_yscale('linear')
        ax_2d.set_ylim([0, FMAX])    
    ax_2d.set_xlim([R_MIN, R_MAX])
    ax_2d.set_xlabel('r [m]')    
    ax_2d.set_ylabel('F [N]')

    plt.show()


interact(plot_coulomb_q, 
         q1 = widgets.FloatSlider(value=Q_MAX, min=Q_MIN, max=Q_MAX, step=0.1),
         q2 = widgets.FloatSlider(value=Q_MAX/2, min=Q_MIN, max=Q_MAX, step=0.1),
         elev = widgets.FloatSlider(value=0, min=0, max=90, step=5), 
         azim = widgets.FloatSlider(value=270, min=0, max=360, step=5),
         n_trace = widgets.IntSlider(value=5, min=1, max=100, step=1),
         show_analytical = widgets.Checkbox(value=False, description='Analytical solution', disabled=False, indent=True)
)

interact(plot_coulomb_r, 
         r = widgets.FloatSlider(value=R_MIN, min=R_MIN, max=R_MAX, step=R_MIN),
         n_trace = widgets.IntSlider(value=5, min=1, max=100, step=1),
         show_analytical = widgets.Checkbox(value=False, description='Analytical solution', disabled=False, indent=True),
         transform_y_axis = widgets.Checkbox(value=False, description='Scale with sqrt(F)', disabled=True, indent=True)  # todo: enable this feature
)

interactive(children=(FloatSlider(value=1.0, description='q1', max=1.0), FloatSlider(value=0.5, description='q…

interactive(children=(FloatSlider(value=0.1, description='r', max=1.0, min=0.1), IntSlider(value=5, descriptio…

<function __main__.plot_coulomb_r(r, n_trace, show_analytical, transform_y_axis)>

## 1.1 introduction
Kinematics tells the story of motion without asking why the motion happens. In one dimension the story unfolds along a single straight line, typically the horizontal $x$‑axis or the vertical $y$‑axis. A description is not complete until we declare a **reference frame**, choose an **origin**, and decide which direction is **positive**. These choices are part of our model; they do not change the physics but they do determine the signs that appear in every equation. With the conventions fixed, **position** $x(t)$ names a location at time $t$, **displacement** $\Delta x = x - x_0$ records the signed change in position between an initial instant $t_0$ and a later instant $t$, and the **elapsed time** $\Delta t = t - t_0$ sets the window of motion we are summarizing. Throughout this chapter we work in everyday conditions where Galilean ideas apply: different inertial observers add relative velocities and agree on the same laws when they use consistent coordinates.

A reference frame is more than a label on a drawing. It encodes the operational meaning of “at rest” and “moving.” The same jogger can have zero velocity in the frame that runs with them and a nonzero velocity in the roadside frame; what matters is that once a frame is chosen, all quantities—positions, velocities, accelerations—are interpreted with respect to that frame. Clear setup at the start prevents sign errors and contradictory statements later.



## 1.2 reference frames
A reference frame is the stage on which motion is described. It consists of an observer, an origin, a set of coordinate axes, and a clock. Every statement about position, velocity, or acceleration is relative to the chosen frame. In everyday life we implicitly adopt the Earth’s surface as our frame, but other choices are equally valid. A passenger walking inside a moving train has a small velocity relative to the train but a large velocity relative to the ground. Both descriptions are correct because they use different frames. Switching between frames means applying transformation rules: in non‑relativistic mechanics, velocities simply add or subtract depending on direction. What never changes are the physical coincidences: both observers agree on whether the passenger reached the door at a given instant, even if they disagree on the passenger’s velocity while doing so. Being explicit about the reference frame and the positive axis choice is essential to avoid sign errors and to make clear which motion is being analyzed.


## 1.2 average velocity
Average velocity condenses an interval of motion into one signed number,
$$\bar v = \frac{\Delta x}{\Delta t}.$$
The numerator is a displacement, not the total path length. That distinction makes average velocity sensitive to direction reversals: an out‑and‑back segment can have a small or even zero $\bar v$ despite a long path. On a position–time graph, $\bar v$ is the slope of the straight line (secant) connecting $(t_0, x_0)$ to $(t, x)$. If the graph is a straight line over an interval, the motion in that interval has constant velocity; if the line bends, the average velocity depends on the chosen window. Modeling always includes the judgment “which interval is physically relevant?”—a whole commute, only the freeway portion, or the dwell time at a stoplight.

Average **speed** is different: it is the total distance traveled divided by the elapsed time. It cannot be negative and it equals $|\bar v|$ only if the motion never reverses direction during the interval.

## 1.3 instantaneous velocity
Instantaneous velocity captures the speed and direction “right now” by taking the average over ever‑smaller time windows,
$$v(t) = \lim_{\Delta t\to 0}\frac{\Delta x}{\Delta t} = \frac{dx}{dt}.$$
Geometrically, $v(t)$ is the slope of the tangent to the $x(t)$ curve. The sign of $v$ encodes direction along the chosen axis: positive means motion in the positive direction, negative means motion in the opposite direction. A constant $v$ gives a straight line $x(t)$; a changing $v$ produces curvature whose steepness hints at later acceleration. In lab data, $v(t)$ is often estimated from finite differences or from the derivative of a smooth fit to noisy position measurements.

Because $v$ is a derivative, idealizations such as “instantaneous reversal” or “sharp impact” are handled as very short transitions relative to the time scale of interest. Piecewise definitions—coast, brake, pause—are natural and map cleanly onto plots and formulas.

## 1.4 speed vs. velocity; distance vs. displacement
**Velocity** is a signed quantity that carries direction information; **speed** is its magnitude and is always nonnegative. Likewise, **displacement** is a signed change in position, whereas **distance traveled** is total path length accumulated over time. These pairs answer different questions. Displacement compares beginning and end only; distance remembers all the wandering between. The magnitude of the average velocity generally differs from the average speed unless the motion is monotonic along the line. When reading a problem, decide up front whether it asks for a magnitude (speed, distance) or a signed quantity (velocity, displacement). That single choice eliminates most ambiguities.

## 1.5 velocity as a vector (in 1D)
Velocity is fundamentally a vector. In one dimension, its direction reduces to a sign, but it is still helpful to write $\vec v = v\,\hat\imath$ (horizontal) or $\vec v = v\,\hat\jmath$ (vertical). This notation keeps direction explicit and makes statements like “$\vec a$ is opposite to $\vec v$” precise. It also reminds us that changing the coordinate choice—moving the origin or flipping the positive direction—must be applied consistently to position, velocity, and acceleration together. The physics does not change when coordinates do; only the algebraic signs do.

## 1.6 acceleration
Acceleration quantifies how velocity changes with time. Over a finite interval,
$$\bar a = \frac{\Delta v}{\Delta t},$$
and at an instant,
$$a(t)=\frac{dv}{dt}.$$
On a velocity–time graph, the slope equals the instantaneous acceleration; on a position–time graph, growing curvature signals a changing velocity. A positive $a$ means that $v$ trends more positive with time; a negative $a$ means that $v$ trends more negative. Crucially, **acceleration and velocity need not point in the same direction**. An object moving in the positive direction can be slowing down if $a<0$, and an object moving in the negative direction can be speeding up if $a<0$ as well—the sign of $a$ relative to $v$ is what decides whether the **speed** is increasing or decreasing.

Because real motions are often multi‑phase, it is natural to treat acceleration as piecewise constant: accelerate off the line, cruise, then brake uniformly. Each phase can be described cleanly, and the pieces are stitched together by matching $x$ and $v$ at the boundaries.

## 1.7 deceleration (slowing down)
“Deceleration” is ordinary language for “the magnitude of the velocity is decreasing.” In vector terms, deceleration occurs when $\vec a$ points opposite to $\vec v$. Since we use signs to encode direction, we do not need a special symbol for deceleration: we simply check whether $a$ and $v$ have opposite signs at that instant. This prevents the common mistake of equating “negative acceleration” with “slowing down.” Negative $a$ means “acceleration directed opposite to the chosen positive direction,” nothing more; whether the motion slows or speeds up depends on the sign of $v$ as well.

## 1.8 motion at constant acceleration
When acceleration is constant, the calculus definitions integrate into a compact toolkit that connects position, velocity, time, and acceleration:
$$v(t)=v_0+at, \qquad x(t)=x_0+v_0t+\tfrac12at^2, \qquad v^2=v_0^2+2a\,(x-x_0), \qquad \bar v=\tfrac12(v_0+v).$$
These relations are valid only under the constant‑$a$ assumption. The first tells us that $v(t)$ is a straight line whose slope is $a$. The second says that $x(t)$ contains a linear “drift” term from any initial velocity plus a quadratic term from the constant slope of $v(t)$. The third eliminates time and is especially useful when only speeds and positions are known; it is the algebraic version of computing the area under the $v(t)$ line. The last identity states that, for constant $a$, the average velocity over any interval equals the midpoint of the initial and final velocities. Graphs make these statements transparent and offer quick checks on sign and scale.

Modeling judgment matters here: constant acceleration is an idealization. It is excellent for motion under a steady thrust or a uniform braking force and for vertical motion near Earth’s surface when air resistance is negligible. It is not appropriate once drag forces dominate or when the driving force varies rapidly in time.

## 1.9 freely falling objects
Near Earth’s surface and in the absence of significant air resistance, all objects accelerate downward with the same magnitude $g\approx 9.81\,\text{m/s}^2$. You may choose “up” as positive so that $a=-g$, or “down” as positive so that $a=+g$; either convention is fine if used consistently. The universality of this acceleration is striking: light and heavy objects share the same $g$ in vacuum. Historically, creating evacuated conditions made this visible—think of Otto von Guericke’s pioneering vacuum demonstrations in Magdeburg that dramatized the effects of removing air. In our one‑dimensional framework, free fall is just constant‑$a$ motion with $a=\pm g$ and $x$ relabeled as $y$ for vertical height.

Two practical notes keep the model honest without complicating it. First, $g$ varies slightly with latitude and elevation, but the standard value suffices for most purposes. Second, air resistance grows with speed and cross‑section; over long falls or for very light objects it cannot be ignored and eventually leads to **terminal velocity**, where the net acceleration approaches zero. We postpone quantitative drag to later chapters so that the core kinematic ideas can stand on their own here.

## 1.10 how to solve problems
Physics is not a collection of equations to be memorized; it is a way of reasoning about motion. A good approach to kinematics problems is structured and consistent. The following steps provide a reliable strategy, adapted directly from the book:

1. **Read and reread the problem carefully.** Make sure you understand what is asked before trying to solve it. Identify whether the question is about position, velocity, acceleration, or time.

2. **Decide what object or system to analyze and what time interval to consider.** Often it is convenient to choose the initial time as $t=0$ to simplify later expressions.

3. **Draw a diagram.** Sketch the situation, including the object, a coordinate axis, the origin, and arrows for velocity or acceleration if appropriate. Choose which direction is positive; rightward or upward are common, but any consistent choice is valid.

4. **List the knowns and unknowns.** Translate language into physics terms: “starting from rest” means $v_0=0$, “dropped” means $v_0=0$ and $a=g$ downward, “comes to a stop” means $v=0$.

5. **Decide which principles apply.** Think about whether constant acceleration is a reasonable assumption. For example, motion under uniform gravity near Earth’s surface is constant acceleration, whereas braking with air resistance is not.

6. **Select the appropriate equations or definitions.** Use the constant‑acceleration kinematic relations only if $a$ is constant. If not, consider graphical reasoning or calculus. When several equations might apply, start by writing them symbolically before inserting numbers.

7. **Solve algebraically for the desired unknown.** Rearranging before plugging in numbers keeps the logic clear and avoids calculator mistakes.

8. **Carry out the calculation.** Keep one or two extra digits through intermediate steps. Round the final answer to the correct number of significant figures and include units.

9. **Check the result.** Does the sign make sense? Are the magnitudes plausible compared with everyday experience? A quick order‑of‑magnitude check using powers of ten often helps. Discard “unphysical” solutions (such as negative times) that fall outside the situation you chose.

Following these steps makes problem solving a process of translating words into physics, applying consistent rules, and then interpreting the result back in plain language.
Effective kinematics problem solving is organized, graphical, and explicit about assumptions. Begin with a quick sketch of the situation and choose a clear axis with marked origin and positive direction. Identify the start and end instants for the interval you will analyze, and translate words into symbols: “from rest” means $v_0=0$, “at constant speed” means $a=0$, “after” signals an elapsed time, and “dropped” means $v_0=0$ in vertical motion. List the knowns (with units) and the unknowns you seek. Decide which regime applies: constant velocity ($x=x_0+vt$), constant acceleration (use the constant‑$a$ toolkit), or variable acceleration (use graphs and, when available, calculus). When motion is piecewise, split it into phases and match the position and velocity at each boundary.

Use plots as a universal cross‑check. On $x(t)$, slopes are velocities; on $v(t)$, slopes are accelerations and areas give displacements. Keep units consistent and meaningful throughout, carry a guard digit or two in intermediate calculations, and round only at the end. Always perform a reality check: verify sign and direction, compare the magnitude with experience, and test simple limits such as $t\to 0$ and $a\to 0$ to see if the expression behaves sensibly. Quadratic relations frequently yield two times; retain only the solution that lies in the physical time window you defined. With these habits, one‑dimensional motion problems become straightforward translations from a well‑posed sketch to a compact algebraic description and back to clear physical interpretation.
